<a href="https://colab.research.google.com/github/SebastienBienfait/L2C-Data-managment/blob/main/Data_Roles_job_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Job Search Script for  and JobsInKent.   

###Intro:
This script accepts user input to search the websites, Reed.co.uk and jobsinkent.com. The user can specify a search term, center of search and radius of search. The user can then specify a filter term which will only return jobs with that term included job title.   

If users wish to save results of the search as a .csv file they can do so in their Google Drive.   

Further down in the script, the data is split into two catagories. Jobs that are posted by a recruitment agency and jobs that are posted by direct employers. An initial list of know recruites is taken from GitHub but it is not expanist*. The list can be added to by direct user imput if the decide any in the list of direct employers are actually recruiters. The updated list can then be saved to Google Drive and used in subsequent searches using this script.

Importing libaries and defining base jobsite urls

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'  ##<- this is really annoying, I WANT to copy the DF.
import numpy as np
import datetime
import matplotlib.pyplot as plt
from google.colab import files
from google.colab import drive

###User Input Function:    
To use this programe, input your search terms in the box below. 

Job Search Term: You can include commers, spaces and multiple words.

Center of Search, use format: 'Town, County' if the center you are searching for is not unique. If it is unique within the UK, 'County' is unnecessary. Or you can just use 'County' as your search center.

--NB: for JobsInKent, the search is limited to 'Kent' by default.

Radius: enter any valid integer. The closest valid radius for each site will be picked. If you are equidistance between two values, the lower value will be choesen.

Filter Term: This will filter the final database to only contain jobs with the Filiter Term in the title: ie 'Data' will filter keep 'Data Developer' but not 'Software Developer'. To not filter at all, type 'no' instead.

--NB: the filter term is case sensitve, 'data' will not pick up 'Database'.

Specify a folder to save in: To save the DataFrame in the folder MyDrive/Collab Notebooks/Folder 1, enter 'Collab Notebooks/Folder 1' when asked.

In [2]:
def user_imput():
  search_term = input("Job Search Term (commer, spaces are allowed): " )
  search_center = input("Center of Search (Town, County): ") #town, county
  search_radius = int(input("Search Radius (Miles): " ))

  filter_term = input("Filter Term: ")
  if filter_term.lower() == "no":
    filter_term = None

  save_state = False
  query_save = input("Would you like to save and download the results of your search? (yes/no) ")
  if query_save.lower() == "yes":
    save_state = True
    save_path = input("Specify a folder in My Drive to save in (or leave blank): ")
  else: 
    save_state = False
    save_path = ""

  recruiters_list_test = input("Do you have a list of known recruiters saved in your Drive? (yes/no): ")

  return search_term, search_center, search_radius, filter_term, save_path, recruiters_list_test

search_term, search_center, search_radius, filter_term, save_path, recruiters_list_test = user_imput()


Job Search Term (commer, spaces are allowed): Data
Center of Search (Town, County): Ashford, Kent
Search Radius (Miles): 30
Filter Term: Data
Would you like to save and download the results of your search? (yes/no) yes


KeyboardInterrupt: ignored

This function finds the closest (rounded down) search radius to the on submitted.

In [ ]:
def find_valid_radius(radius,valid_list):
  valid_test = np.array([],dtype="int")

  for radi in valid_list:
    valid_test = np.append(valid_test,[abs(radi-radius)])
  return_radius = valid_list[np.argmin(valid_test)]
  return return_radius

This takes the search terms above and creates the URL

In [ ]:
def creat_urls(search_term, search_center, search_radius):
  jobsinkent_search_str = search_term.replace(",","%2C").replace(" ","+")
  reed_search_str = search_term.replace(",","%2C").replace(" ","%20")

  reed_center_str = search_center.replace(", ","%2C-")

  jobsinkent_valid_radius = np.array([1,2,5,10,20,30,40],dtype="int")
  jobsinkent_radius = find_valid_radius(search_radius, jobsinkent_valid_radius)

  reed_valid_radius = np.array([0,1,3,5,10,15,20,30,50],dtype="int")
  reed_radius = find_valid_radius(search_radius, reed_valid_radius)

  jobsinkent_url = "https://jobsinkent.com/search?q=" + jobsinkent_search_str + "&pl=1" + "&r=" + str(jobsinkent_radius)
  reed_url = "https://www.reed.co.uk/jobs?keywords=" + reed_search_str+"&location=" + reed_center_str + "&proximity=" + str(reed_radius)
  print(jobsinkent_url)
  print(reed_url)

  return reed_url, jobsinkent_url

In [ ]:
reed_url, jobsinkent_url = creat_urls(search_term, search_center, search_radius)

max_job_call = 2000 #reed has a limit of 2000 job search api calls per hour.

General Functions that are used in both searches.

In [ ]:
#returns the full HTML of a page.
def get_html(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

#This cleans the dataframes when they have been created.
def general_clean(df,site,filter_term):
  df.drop_duplicates(inplace=True)
  df["salary"].replace(np.nan, '', regex=False,inplace=True) #not all companies have listed a salary and so NaN is sometimes returned
  df["company"] = df["company"].str.lstrip(" ").str.rstrip(" ") #some companies have excess whitespaces around their names
  df.dropna(inplace=True) #just incase the max of 2000 jobs is bypassed

  if filter_term != None:
    df = df[df["job_title"].str.contains(filter_term)]

  df["date_found"] = datetime.date.today()
  df["advertised_on"] = site
  return df

#Reed Job Search
---

This function finds the job ID for all jobs displayed on each page url from reed.co.uk


In [ ]:
def reed_jobs_iterate(page_url):
  site_html = get_html(page_url)
  results = site_html.find(class_="col-sm-8 col-md-9 results-container")

  id_list = np.array([])
  job_cards = results.find_all(class_="job-result-card")

  for job in job_cards: #finds the jobs, gets it's ID, returns the id
      job_id = int(job["id"].split("jobSection")[1]) #jobSection48529572 ect...
      
      id_list = np.append(id_list,[job_id])
  return id_list

This finds the total jobs found in the search

In [ ]:
def find_reed_total_jobs(site_html,max_job_call):
  max_page_requests = int(max_job_call/25) 

  ###
  total_jobs_text = site_html.find(class_="col-sm-11 col-xs-12 page-title").text # '\n' '\r' '\n' x,xxx\r\n  Data Jobs near Ashford       '\n'....
  total_jobs_text = total_jobs_text.replace("\n","").replace("\r","").replace(",","") #          xxxx         Data Jobs near Ashford       .       

  total_jobs = int(total_jobs_text.split("Data")[0].strip(" "))
  total_pages = int(np.ceil(total_jobs/25))

  #limits the requests to only 2000
  if total_pages > max_page_requests:
    total_pages = max_page_requests


  return total_jobs, total_pages


The function scrapes website for the job IDs of every job that has been found by the search criteria.


In [ ]:
def reed_scrape(base_url,max_job_call):
  soup = get_html(base_url)
  total_id_list = np.array([],dtype=int)

  #finds out how many more pages to scrape
  total_jobs, total_pages = find_reed_total_jobs(soup,max_job_call)

  print("total jobs found: ", total_jobs)
  print("total pages: ", total_pages)
  print("pages to search: ",total_pages)

  for page_no in np.arange(1,total_pages+1): 
    page_url = base_url+"&pageno="+str(page_no)
    page_id_list = reed_jobs_iterate(page_url)
    total_id_list = np.append(total_id_list,[page_id_list])

    #user feedback
    print("page ",page_no, " processed")
  return total_id_list


This gives the job IDs to the Reed API to return job information based on the job ID

In [ ]:
def call_reed_api(id_list):
  base_url = "https://www.reed.co.uk/api/1.0/jobs/"
  api_url = ""
  job_df = pd.DataFrame()

  #user feedback
  print("Retriving job information...")

  for job_id in id_list:
    api_url = base_url+str(job_id)
    post_request = requests.get(api_url, auth=("96a7ec49-549c-4529-b9d2-fa3059a437b3",""))
    json_data = post_request.json()

    new_row = pd.json_normalize(json_data)
    job_df = job_df.append(new_row,ignore_index=True)
  return job_df

Calls the scraping function and the api function in turn ~7-8mins to run

In [ ]:
reed_id_list = reed_scrape(reed_url,max_job_call)
reed_id_list = reed_id_list.astype("int") #it really didnt want to save the array as an int

full_reed_df = call_reed_api(reed_id_list)
display(full_reed_df.info())

Cleaning the Reed DF

In [ ]:
def clean_reed(df_in): #this filters out the necessary rows and renames them so the cleaning function works.
  df = df_in[["jobTitle", "employerName", "salary", "contractType", "locationName",  "jobUrl"]]

  df.rename(columns={"jobTitle":"job_title", "employerName": "company", "contractType":"contract", "locationName":"location",  "jobUrl":"job_url"}, inplace=True)
  return df

reed_df = clean_reed(full_reed_df)
reed_df = general_clean(reed_df,"https://www.reed.co.uk",filter_term)
display(reed_df.shape)
display(reed_df.head(5))

#JobsInKent Job Search
---


This function returns a DF containing all of the relevent jobs found on 1 page

In [ ]:
def jobsinkent_iterate(url,job_limit):
  jobs_df = pd.DataFrame(data=[], columns = ["job_title", "company", "salary", "contract", "location","job_url"])
  site_html = get_html(url)
  results = site_html.find(class_="mt-2 lg:mt-0")

  #this is finding an additional 20 jobs per page that are not visible on the site itself.
  job_cards = results.find_all(class_="flex mb-2 border-black-900 rounded-md border-x-4 sm:rounded-lg")


  #[:job_limit] is required as non relevent jobs were being found in the site_html with the job_cards class after the 20~ jobs that were being show on the site.
  # So I had to find how many jobs were being show in each page first and then limit the loop to only iterate up to that number of jobs.
  for job in job_cards[:job_limit]:

    job_title_url = job.find("a", class_="text-blue-700 visited:text-grey-200 hover:underline") 
    job_title = job_title_url.text
    job_url = job_title_url["href"]

    company_name = job.find(class_="mt-0 mb-2 max-w-2xl text-sm text-black").text 

    #This info was all on the fouth box class and needed to be seperated further for the relevent information to be extracted.
    info_list = job.find_all(class_="col-span-12") 
    salary = clean_bloat(info_list[0].text)
    contract = clean_bloat(info_list[1].text)
    location = clean_bloat(info_list[2].text)

    new_row = {"job_title":job_title, "company":company_name, "salary":salary, "contract":contract, "location":location,"job_url":job_url}
    jobs_df = jobs_df.append(new_row,ignore_index = True)
  return jobs_df

String manipulation funtions.

In [ ]:
#removes '\n' and spaces from from names.
def clean_bloat(text):
  strip1 = text.replace("\n","").replace("\xa0","")
  strip2 = strip1.strip(" ")
  return strip2

#extracts information about numbers of jobs 
def return_jobs_pages(input_str):
  p_j = input_str.split("of") #[showing 1 to yy] , [xx jobs]
  jobs_per_page = int(p_j[0].split("to")[1]) #[yy]
  total_jobs = int(p_j[1].split("Jobs")[0]) #[xx]
  
  total_pages = int(np.ceil(total_jobs/jobs_per_page))

  return total_jobs, jobs_per_page, total_pages

The main function that calls to find out how many pages are needed to search over and stores the results of each iteration call.

In [ ]:
def jobsinkent_scrape(base_url):
  #initialse the DF that will be used
  jobsinkent_df = pd.DataFrame(data=[], columns = ["job_title", "company", "salary", "contract", "location","job_url"])

  #returns ALL the html from the site
  site_html = get_html(base_url)

  banner = site_html.find("p", class_="text-sm text-gray-700 leading-5") #~6th child within the top banner.
  showing = banner.text # 'Showing 1 to yy of xx Jobs'

  total_jobs, jobs_per_page, total_pages = return_jobs_pages(showing)

  for page_no in range(1,total_pages+1):
    #this if statment is required due to the issues mentioned above about job limits. It sets the jobs_limit variable to however many jobs there are displayed (visibly) on the page.
    if page_no == total_pages:
      jobs_limit = total_jobs%jobs_per_page
    else:
      jobs_limit = jobs_per_page

    page_url = base_url + "&page=" + str(page_no)
    new_df = jobsinkent_iterate(page_url, jobs_limit)
    jobsinkent_df = jobsinkent_df.append(new_df, ignore_index=True)
  
    #user feedback  
    print("page ",page_no, " processed")

  return jobsinkent_df
  
jobsinkent_df = jobsinkent_scrape(jobsinkent_url)
display(jobsinkent_df.head(5))

Cleans the jobsinkent_df to be used later.

In [ ]:
jobsinkent_df = general_clean(jobsinkent_df,"jobsinkent.com",filter_term)

#Combining both results
---


Combines the two DF's from JobsInKent

In [ ]:
data_jobs_df = pd.DataFrame()

data_jobs_df = data_jobs_df.append([jobsinkent_df,reed_df],ignore_index=True)
display(data_jobs_df.head())

In [ ]:
def importing_recruiters(recruiters_list_test,save_folder):
  
  if recruiters_list_test == "yes":
    drive.mount('/content/drive')
    list_of_recruiters = pd.read_csv("/content/drive/MyDrive/" + save_folder + "/recruiters_list.csv")
    display(list_of_recruiters)
    drive.flush_and_unmount
  else:
    list_of_recruiters = pd.read_csv("https://raw.githubusercontent.com/futureCodersSE/data-roles/main/datasets/recruiters_list.csv")

  return list_of_recruiters

list_of_recruiters = importing_recruiters(recruiters_list_test,save_folder)


This returns a **unique** list of recruiters and compaies who are advertiseing data roles

In [ ]:
def find_company_lists1(initail_df):
  recruiters_list = list_of_recruiters["company"]
  data_jobs_employer_names = initail_df.drop_duplicates(subset = "company")

  companies_df = data_jobs_employer_names.drop( data_jobs_employer_names[data_jobs_employer_names["company"].isin(recruiters_list) == True ].index)
  companies_list = companies_df["company"]

  return companies_list, recruiters_list

companies_list, recruiters_list = find_company_lists1(data_jobs_df)


This function takes the list of known Recruitment Agencies from GitHub and splits the DataFrame and return the list of know Recruitment Agencies and Companies that are in that DataFrame

In [ ]:
def find_company_lists():
  data_jobs_employer_names = data_jobs_df.drop_duplicates(subset = "company") #data_jobs_df["company"].unique()

  #seperating out all recruitment companies from the list
  companies_df = data_jobs_employer_names.drop( data_jobs_employer_names[ data_jobs_employer_names["company"].str.contains("Recruit")==True].index)

  #This requires going through each company and deciding if they are a recruitment agency or not if they dont have 'Recruit' in their name and adding them to the list.
  additional_recruiter_list = ["Reed","OnetoOne Personnel","Huntress","MW Appointments","P3 Search & Selection","Manpower - Ashford","Morgan McKinley","Brook Street","McGregor Boyall","SAGA","Office Angels","REED",
                            "Commercial Services Interim & Executive Search","Harnham","Connect2Staff","Senitor Associates","Zorba Consulting Limited","GerrardWhite","Lorien","Morgan Law", "Academics","eFinancialCareers",
                            "Commercial Services HR", "OBJECTIVE HR LTD" ]
  it_specialits = ["Senitor Associates","Zorba Consulting Limited","GerrardWhite","Lorien","eFinancialCareers"]

  companies_df = companies_df.drop( companies_df[companies_df["company"].isin(additional_recruiter_list) == True ].index)
  companies_list = companies_df["company"]

  recruiters_df1 = data_jobs_employer_names.drop( data_jobs_employer_names[ data_jobs_employer_names["company"].str.contains("Recruit")==False].index) #drop 'Recruit' is not in name
  recruiters_df2 = data_jobs_employer_names.drop( data_jobs_employer_names[data_jobs_employer_names["company"].isin(additional_recruiter_list) == False ].index) #drop if company is not in 'additional_drop_list'

  recruiters_df = recruiters_df1.append(recruiters_df2,ignore_index=True) #combine the two sets of lists
  recruiters_list = recruiters_df["company"]

  return companies_list, recruiters_list

companies_list, recruiters_list = find_company_lists()

This function prints the list of Companies for the user to manually check if they are Recruitment Agencies. By entering in the index of any suspected recruitment agancies, these are dropped from companies list and added to the recruiters list.   

User Input: "2,14,20..." or "2, 14, 20..."

In [ ]:
companies_list = companies_list.reset_index()["company"]

full_recruiters_series = pd.Series([])

def find_more_recruiters(companies_list, recruiters_found):
  all_checked = False
  n = 0 #counter
  while all_checked != True:
    list_max = len(companies_list)
    print("companies to check: ",list_max)

    print(n)
    while n+20 <= list_max:

      for i in range(n,n+20):   #company in companies_list[n:n+5]:
        print(i, companies_list[i])

      recruiter_index_str = input("Enter the Index of a Recruitment Agencies on this list or type 'no' to contiune ")

      if recruiter_index_str.lower() == "no":
        n += 20
      else:
        recruiter_index_list = recruiter_index_str.replace(" ","").split(",")
        recruiter_index_ints = [eval(x) for x in recruiter_index_list]

        drop_index_S = pd.Series( [ companies_list.loc[recruiter_index_ints] ] )
        print(drop_index_S[0])

        recruiters_found = recruiters_found.append(drop_index_S[0], ignore_index=True)
        companies_list.drop(recruiter_index_ints, inplace = True)
        companies_list = companies_list.reset_index()["company"]
        list_max -= len(drop_index_S[0])
      ###########
      #End of Inner While Statment.
      ###########

    for i in range(n,list_max):
      print(i, companies_list[i])

    recruiter_index_str = input("Enter the Index of a Recruitment Agencies on this list or type 'no' to contiune ")

    if recruiter_index_str == "no":
      all_checked = True
      print("All Companies Have Been Checked!")
      print("droped companies: ")
      display(recruiters_found)
      ###########
      #End of Outer While Statment.
      ###########
    else:
        recruiter_index_list = recruiter_index_str.replace(" ","").split(",")
        recruiter_index_ints = [eval(x) for x in recruiter_index_list]

        drop_index_S = pd.Series( [ companies_list.loc[recruiter_index_ints] ] )
        print(drop_index_S[0])

        recruiters_found = recruiters_found.append(drop_index_S[0], ignore_index=True)
        companies_list.drop(recruiter_index_ints, inplace = True)
        companies_list = companies_list.reset_index()["company"]
        list_max -= len(drop_index_S[0])


  return companies_list, recruiters_found


companies_list, recruiters_found = find_more_recruiters(companies_list, full_recruiters_series)



This funtion combines the list of known Recruiment Agencies and the list found by the user. To update the .CSV file on GitHub. Download the file and upload it to https://github.com/futureCodersSE/data-roles/tree/main/datasets under the same name. You can rerun the previouse few cells to check that the updated list is effective.

In [ ]:
def update_recruiters_list(recruiters_list, recruiters_found):
  recruiters_list = recruiters_list.append(recruiters_found,ignore_index=True)
  display(recruiters_list)

  drive.mount('/content/drive')
  recruiters_list.to_csv("recruiters_list.csv",index = 0)
update_recruiters_list(recruiters_list, recruiters_found)

This takes the lists of unique compaies and recruiters and returns a DataFrame with **all** the roles they have advertised.

In [ ]:
def find_company_df():
  company_df = data_jobs_df.drop( data_jobs_df[data_jobs_df["company"].isin(recruiters_list) == True ].index) #drop if company is a recruiter
  recruiters_df = data_jobs_df.drop( data_jobs_df[data_jobs_df["company"].isin(companies_list) == True ].index) #drop if company is not a recruiter

  #For readability.
  company_df.reset_index(drop=True, inplace = True)
  recruiters_df.reset_index(drop=True, inplace = True)

  return company_df, recruiters_df

company_df, recruiters_df = find_company_df()
  

Saves the DataFrames with a name based on the date searched. If you run the search multiple times, the newer search file will overwrite the older search file.

In [ ]:
#saveing files names of search based on current date.
date_today = datetime.date.today()
recruiters_fn = str(date_today)+"_recruitment_company_df.csv"
company_fn = str(date_today)+"_data_employeer_df.csv"

#'opens' the chosen folder in drive for files to be saved in.
def mount_drive(data_path):
  drive.mount('/content/drive/', force_remount=True)
  project_dir = "/content/drive/MyDrive/" + data_path #--working parth
  return project_dir

  
def unmount_drive():
  drive.flush_and_unmount()
  print('Drive Unmounted')

project_dir = mount_drive(save_path)

Checks if you have choesn to save your work to your Drive.

In [ ]:
if query_save.lower() == "yes":
    save_state = True
    
    recruiters_list.to_csv(project_dir +"/recruiters_list.csv")
    recruiters_df.to_csv(project_dir + recruiters_fn)
    company_df.to_csv(project_dir + company_fn)

    unmount_drive()
  else: 
    save_state = False

Prints confirmation of successful save and the file name/location.

In [ ]:
if save_state == True:
  save()
  download()
  print("Saved and Printed")
else:
  print("You did not save the DF")

print("Search Terms:")
print(search_term)
print(search_center)
print(search_radius)